In [1]:
import pandas as pd
import geopandas as gpd

In [20]:
gaybizDf = pd.read_pickle('/Users/michaelcriste/Documents/GitHub/informal-queer-spaces/data/newgaybiz_clean.pandas')
gaybizDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1719 entries, 0 to 1718
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   business_id       1719 non-null   object
 1   gay_text          1719 non-null   object
 2   gay_text_len      1719 non-null   int64 
 3   gay_counts        1719 non-null   object
 4   gay_unique_count  1719 non-null   int64 
 5   known             1719 non-null   bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 82.3+ KB


In [18]:
gaybizDf.head()

,business_id,gay_text,gay_text_len,gay_counts,gay_unique_count,known
0,wPi7p5fZyGTOjDU28IlTqA,"[gay, gay, lgbtq, gay, gay, gay, gay, gay, gay...",222,"{'gay': 178, 'lgbtq': 5, 'twinks': 1, 'lesbian...",13,True
1,FhwTEGqdir75uyXZqWqxlw,"[gay, gay, gay, gay, gay, gay, gay, gay, gay, ...",169,"{'gay': 153, 'gays': 5, 'queen': 9, 'homophobi...",5,False
2,ZH41CKIlAZrrQD-DMV2TWA,"[drag, drag, drag, drag, drag, drag, drag, dra...",165,"{'drag': 130, 'queens': 10, 'gay': 12, 'homose...",9,True
3,4FHXsFTyvvxc-pISdYkoeQ,"[lesbians, lesbian, lesbians, gay, lesbians, l...",105,"{'lesbians': 16, 'lesbian': 40, 'gay': 31, 'qu...",9,False
4,IORvbG2rAIVJ-1d148cCmg,"[drag, gay, gay, queer, gay, gay, gay, gay, qu...",90,"{'drag': 10, 'gay': 54, 'queer': 9, 'queen': 2...",10,True


In [22]:
phillybizDf = pd.read_pickle('/Users/michaelcriste/Documents/GitHub/informal-queer-spaces/data/bizPhil.pandas')
phillybizDf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 44840 entries, 3 to 150339
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   business_id   44840 non-null  object  
 1   name          44840 non-null  object  
 2   address       44840 non-null  object  
 3   city          44840 non-null  object  
 4   state         44840 non-null  object  
 5   postal_code   44840 non-null  object  
 6   latitude      44840 non-null  float64 
 7   longitude     44840 non-null  float64 
 8   stars         44840 non-null  float64 
 9   review_count  44840 non-null  int64   
 10  is_open       44840 non-null  int64   
 11  attributes    40920 non-null  object  
 12  categories    44802 non-null  object  
 13  hours         37236 non-null  object  
 14  geometry      44840 non-null  geometry
dtypes: float64(3), geometry(1), int64(2), object(9)
memory usage: 5.5+ MB


In [23]:
phillybizDf.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,geometry
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",POINT (-75.15556 39.95551)
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",POINT (-75.47166 40.33818)
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",POINT (-75.14323 39.95395)
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",POINT (-75.16257 39.94322)
25,PSo_C1Sfa13JHjzVNW6ziQ,Indian Walk Veterinary Center,662 Durham Rd,Newtown,PA,18940,40.273493,-74.987496,5.0,15,1,None,"Pet Services, Pet Groomers, Pets, Veterinarians","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",POINT (-74.98750 40.27349)


In [26]:
joinedDf = pd.merge(gaybizDf, phillybizDf, how='left', on='business_id')
joinedDf.head()

,business_id,gay_text,gay_text_len,gay_counts,gay_unique_count,known,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,geometry
0,wPi7p5fZyGTOjDU28IlTqA,"[gay, gay, lgbtq, gay, gay, gay, gay, gay, gay...",222,"{'gay': 178, 'lgbtq': 5, 'twinks': 1, 'lesbian...",13,True,Woody's,202 S 13th St,Philadelphia,PA,19107,39.949064,-75.162410,3.0,325,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Bars, American (Traditional), Gay...","{'Monday': '19:0-2:0', 'Tuesday': '19:0-2:0', ...",POINT (-75.16241 39.94906)
1,FhwTEGqdir75uyXZqWqxlw,"[gay, gay, gay, gay, gay, gay, gay, gay, gay, ...",169,"{'gay': 153, 'gays': 5, 'queen': 9, 'homophobi...",5,False,Big Gay Ice Cream,1351 South St,Philadelphia,PA,19147,39.943738,-75.165094,3.5,404,0,"{'RestaurantsPriceRange2': '1', 'GoodForMeal':...","Ice Cream & Frozen Yogurt, Party & Event Plann...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:...",POINT (-75.16509 39.94374)
2,ZH41CKIlAZrrQD-DMV2TWA,"[drag, drag, drag, drag, drag, drag, drag, dra...",165,"{'drag': 130, 'queens': 10, 'gay': 12, 'homose...",9,True,Bob & Barbara's,1509 South St,Philadelphia,PA,19146,39.944050,-75.167550,4.0,295,1,"{'RestaurantsGoodForGroups': 'True', 'BikePark...","Lounges, Bars, Nightlife, Dive Bars, Adult Ent...","{'Wednesday': '16:0-2:0', 'Thursday': '16:0-2:...",POINT (-75.16755 39.94405)
3,4FHXsFTyvvxc-pISdYkoeQ,"[lesbians, lesbian, lesbians, gay, lesbians, l...",105,"{'lesbians': 16, 'lesbian': 40, 'gay': 31, 'qu...",9,False,Sisters Restaurant & Nightclub,1320 Chancellor St,Philadelphia,PA,19107,39.948499,-75.163109,3.0,51,0,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","Bars, Restaurants, Dance Clubs, Gay Bars, Nigh...",None,POINT (-75.16311 39.94850)
4,IORvbG2rAIVJ-1d148cCmg,"[drag, gay, gay, queer, gay, gay, gay, gay, qu...",90,"{'drag': 10, 'gay': 54, 'queer': 9, 'queen': 2...",10,True,Voyeur Nightclub,1221 Saint James St,Philadelphia,PA,19107,39.948345,-75.161863,3.0,163,1,"{'RestaurantsPriceRange2': '2', 'NoiseLevel': ...","Nightlife, Dance Clubs, Gay Bars, Bars","{'Tuesday': '23:45-3:0', 'Wednesday': '23:0-3:...",POINT (-75.16186 39.94834)


In [27]:
joinedDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1719 entries, 0 to 1718
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   business_id       1719 non-null   object  
 1   gay_text          1719 non-null   object  
 2   gay_text_len      1719 non-null   int64   
 3   gay_counts        1719 non-null   object  
 4   gay_unique_count  1719 non-null   int64   
 5   known             1719 non-null   bool    
 6   name              1719 non-null   object  
 7   address           1719 non-null   object  
 8   city              1719 non-null   object  
 9   state             1719 non-null   object  
 10  postal_code       1719 non-null   object  
 11  latitude          1719 non-null   float64 
 12  longitude         1719 non-null   float64 
 13  stars             1719 non-null   float64 
 14  review_count      1719 non-null   int64   
 15  is_open           1719 non-null   int64   
 16  attributes        1642 n

In [28]:
joinedDf.head(100)

,business_id,gay_text,gay_text_len,gay_counts,gay_unique_count,known,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,geometry
0,wPi7p5fZyGTOjDU28IlTqA,"[gay, gay, lgbtq, gay, gay, gay, gay, gay, gay...",222,"{'gay': 178, 'lgbtq': 5, 'twinks': 1, 'lesbian...",13,True,Woody's,202 S 13th St,Philadelphia,PA,19107,39.949064,-75.162410,3.0,325,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Bars, American (Traditional), Gay...","{'Monday': '19:0-2:0', 'Tuesday': '19:0-2:0', ...",POINT (-75.16241 39.94906)
1,FhwTEGqdir75uyXZqWqxlw,"[gay, gay, gay, gay, gay, gay, gay, gay, gay, ...",169,"{'gay': 153, 'gays': 5, 'queen': 9, 'homophobi...",5,False,Big Gay Ice Cream,1351 South St,Philadelphia,PA,19147,39.943738,-75.165094,3.5,404,0,"{'RestaurantsPriceRange2': '1', 'GoodForMeal':...","Ice Cream & Frozen Yogurt, Party & Event Plann...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:...",POINT (-75.16509 39.94374)
2,ZH41CKIlAZrrQD-DMV2TWA,"[drag, drag, drag, drag, drag, drag, drag, dra...",165,"{'drag': 130, 'queens': 10, 'gay': 12, 'homose...",9,True,Bob & Barbara's,1509 South St,Philadelphia,PA,19146,39.944050,-75.167550,4.0,295,1,"{'RestaurantsGoodForGroups': 'True', 'BikePark...","Lounges, Bars, Nightlife, Dive Bars, Adult Ent...","{'Wednesday': '16:0-2:0', 'Thursday': '16:0-2:...",POINT (-75.16755 39.94405)
3,4FHXsFTyvvxc-pISdYkoeQ,"[lesbians, lesbian, lesbians, gay, lesbians, l...",105,"{'lesbians': 16, 'lesbian': 40, 'gay': 31, 'qu...",9,False,Sisters Restaurant & Nightclub,1320 Chancellor St,Philadelphia,PA,19107,39.948499,-75.163109,3.0,51,0,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","Bars, Restaurants, Dance Clubs, Gay Bars, Nigh...",None,POINT (-75.16311 39.94850)
4,IORvbG2rAIVJ-1d148cCmg,"[drag, gay, gay, queer, gay, gay, gay, gay, qu...",90,"{'drag': 10, 'gay': 54, 'queer': 9, 'queen': 2...",10,True,Voyeur Nightclub,1221 Saint James St,Philadelphia,PA,19107,39.948345,-75.161863,3.0,163,1,"{'RestaurantsPriceRange2': '2', 'NoiseLevel': ...","Nightlife, Dance Clubs, Gay Bars, Bars","{'Tuesday': '23:45-3:0', 'Wednesday': '23:0-3:...",POINT (-75.16186 39.94834)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PP3BBaVxZLcJU54uP_wL6Q,"[drag, queen, queens, bear, queen, bear, queen...",13,"{'drag': 2, 'queen': 3, 'queens': 2, 'bear': 6}",4,False,Pat's King of Steaks,1237 E Passyunk Ave,Philadelphia,PA,19147,39.933201,-75.159266,3.0,4250,1,"{'RestaurantsReservations': 'False', 'Corkage'...","Italian, American (Traditional), Sandwiches, F...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",POINT (-75.15927 39.93320)
96,FXj02QYnpbLGzR5MGW_LHw,"[bear, bear, bear, bear, bear, bear, bear, bea...",13,{'bear': 13},1,False,Taws Can Do,1530 Locust St,Philadelphia,PA,19102,39.948351,-75.167584,2.5,28,0,"{'BusinessParking': '{'garage': False, 'street...","Art Supplies, Shopping, Cards & Stationery, Lo...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",POINT (-75.16758 39.94835)
97,1yuSWypzGewvhNZSwUlvPw,"[queen, queen, queens, queen, queen, queen, qu...",13,"{'queen': 11, 'queens': 2}",2,False,Ela,627 S 3rd St,Philadelphia,PA,19147,39.940613,-75.147572,4.0,323,0,"{'Ambience': '{'touristy': False, 'hipster': F...","American (New), Restaurants, Bars, Breakfast &...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'...",POINT (-75.14757 39.94061)
98,yW0DwYGKyZjBNOvingjKzw,"[queen, queen, queen, queen, queen, queen, que...",13,{'queen': 13},1,False,Que Rica's,46 A Haddon Ave,Haddon Township,NJ,08108,39.911821,-75.053973,4.5,56,1,"{'Alcohol': 'u'none'', 'GoodForKids': 'True', ...","Latin American, Venezuelan, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ...",POINT (-75.05397 39.91182)


In [29]:
#this is it. Generate pickle of visible and invisible queer business
joinedDf.to_pickle('data/completegaybiz.pandas')